In [14]:
import pandas as pd
import os, pickle

In [4]:
data_dir = os.path.join(os.getcwd(), "Dataset")
music_df = pd.read_csv(os.path.join(data_dir, "Music_Lyrics.csv"), index_col=0)

In [5]:
processed_df = pd.read_csv(os.path.join(data_dir,"emotions_processed_df.csv"))
processed_df = processed_df.rename(columns={"sentiment": "labels", "content": "text"})
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(processed_df['labels'])
le_name_mapping = dict(zip(le.transform(le.classes_), le.classes_))
#print(le_name_mapping)

#print both key and value from mapping 
for key, value in le_name_mapping.items():
    print(key, value)

0 anger
1 boredom
2 empty
3 enthusiasm
4 fear
5 fun
6 happiness
7 happy
8 hate
9 love
10 neutral
11 relief
12 sadness
13 worry


In [8]:
validation_df = music_df.sample(20, random_state=7)

In [10]:
validation_df.to_csv(os.path.join(data_dir, "lyrics_validation_df.csv"))

In [18]:
predicted_df = pd.read_csv(os.path.join(data_dir, "lyrics_validation_df_predict.csv"))

In [19]:
predicted_df = predicted_df[["predict", "lyrics"]]

In [20]:
predicted_df

,predict,lyrics
0,sadness,dream strong remain toss turn night sheet slee...
1,happy,steer right sweet steer right fall steer right...
2,fun,mover party groover mellow super duper break m...
3,sadness,hide feel motion control tear eye poor heart c...
4,sadness,star night bright deep heart texas prairie wid...
5,sadness,know grow cold hearts turn stone break shame w...
6,sadness,time break hearts break dream place wish leaf ...
7,worry,night motorcycle desert speed smash body frien...
8,fun,alabama boombox baby bout little boondock craz...
9,anger,arrogance violence world disarray deal insanit...


In [21]:
#return results from mapping dictionary
def results(result):
    return le_name_mapping[result[0]]

In [22]:
ml_model_list = ["logistic_regression-ovr-c1_emotions.pkl","logistic_regression-multinomial-c1_emotions.pkl","multinomialNB_emotions.pkl","linear_svc_emotions.pkl"]
from sklearn.feature_extraction.text import CountVectorizer
cv = pickle.load(open('cv (selected) emotions.pkl', 'rb'))
for model_dir in ml_model_list:
    model = pickle.load(open(model_dir, "rb"))
    temp_list = []
    for lyrics in predicted_df["lyrics"]:
        lyrics_vect = cv.transform([lyrics]).toarray()
        result = model.predict(lyrics_vect)
        temp_list.append(results(result))
        
    predicted_df[model_dir] = temp_list

In [23]:
predicted_df

,predict,lyrics,logistic_regression-ovr-c1_emotions.pkl,logistic_regression-multinomial-c1_emotions.pkl,multinomialNB_emotions.pkl,linear_svc_emotions.pkl
0,sadness,dream strong remain toss turn night sheet slee...,worry,worry,sadness,fear
1,happy,steer right sweet steer right fall steer right...,love,love,happy,love
2,fun,mover party groover mellow super duper break m...,sadness,fun,sadness,enthusiasm
3,sadness,hide feel motion control tear eye poor heart c...,anger,anger,sadness,anger
4,sadness,star night bright deep heart texas prairie wid...,sadness,sadness,sadness,sadness
5,sadness,know grow cold hearts turn stone break shame w...,sadness,sadness,sadness,worry
6,sadness,time break hearts break dream place wish leaf ...,sadness,sadness,sadness,sadness
7,worry,night motorcycle desert speed smash body frien...,hate,hate,sadness,hate
8,fun,alabama boombox baby bout little boondock craz...,fun,fun,worry,sadness
9,anger,arrogance violence world disarray deal insanit...,hate,hate,sadness,hate


In [27]:
import pandas as pd
import numpy as np
import transformers
import os, warnings, pickle, torch
warnings.filterwarnings('ignore')

In [28]:
def to_check_result(test_encoding, model, device):
    input_ids = torch.tensor(test_encoding["input_ids"]).to(device)
    attention_mask = torch.tensor(test_encoding["attention_mask"]).to(device)
    with torch.no_grad():
        output = model(input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0))
    return np.argmax(output[0].to("cpu").numpy())

In [30]:
model_dir = os.path.join(os.getcwd(), 'models')
models = [(transformers.BertForSequenceClassification, transformers.BertTokenizer, 'bert-base-uncased_v2'),
            (transformers.AlbertForSequenceClassification, transformers.AlbertTokenizerFast, 'albert-large-v1_v2'),
            (transformers.AlbertForSequenceClassification, transformers.AlbertTokenizerFast, 'albert-large-v2'),
            (transformers.DistilBertForSequenceClassification, transformers.DistilBertTokenizerFast, 'distilbert-base-uncased_v2'),
            (transformers.RobertaForSequenceClassification, transformers.RobertaTokenizer, 'roberta-base_v2')]


for model, tokenizer, model_path in models:
    model_class = model
    tokenizer_class = tokenizer
    pretrained_weights = model_path
    saved_dir = os.path.join(model_dir, pretrained_weights)
    # Load pretrained model/tokenizer
    tokenizer = tokenizer_class.from_pretrained(saved_dir)
    model = model_class.from_pretrained(saved_dir)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    # Copy the model to the GPU.
    model.to(device)
    temp_list = []
    for lyrics in predicted_df["lyrics"]:
        predict_emotions = []
        
        # Tokenize lyrics
        encoded = tokenizer(lyrics, padding=True, truncation=True)
        
        # Predict emotion
        predict_emotions.append(to_check_result(encoded, model, device))
        temp_list.append(results(predict_emotions))
    predicted_df[pretrained_weights] = temp_list

In [31]:
predicted_df

,predict,lyrics,logistic_regression-ovr-c1_emotions.pkl,logistic_regression-multinomial-c1_emotions.pkl,multinomialNB_emotions.pkl,linear_svc_emotions.pkl,bert-base-uncased_v2,albert-large-v1_v2,albert-large-v2,distilbert-base-uncased_v2,roberta-base_v2
0,sadness,dream strong remain toss turn night sheet slee...,worry,worry,sadness,fear,neutral,sadness,neutral,worry,fear
1,happy,steer right sweet steer right fall steer right...,love,love,happy,love,love,sadness,neutral,love,happy
2,fun,mover party groover mellow super duper break m...,sadness,fun,sadness,enthusiasm,neutral,sadness,worry,happiness,fun
3,sadness,hide feel motion control tear eye poor heart c...,anger,anger,sadness,anger,sadness,sadness,sadness,anger,sadness
4,sadness,star night bright deep heart texas prairie wid...,sadness,sadness,sadness,sadness,love,sadness,neutral,love,sadness
5,sadness,know grow cold hearts turn stone break shame w...,sadness,sadness,sadness,worry,sadness,sadness,sadness,anger,sadness
6,sadness,time break hearts break dream place wish leaf ...,sadness,sadness,sadness,sadness,sadness,sadness,sadness,sadness,sadness
7,worry,night motorcycle desert speed smash body frien...,hate,hate,sadness,hate,hate,sadness,worry,worry,sadness
8,fun,alabama boombox baby bout little boondock craz...,fun,fun,worry,sadness,neutral,sadness,neutral,fun,happiness
9,anger,arrogance violence world disarray deal insanit...,hate,hate,sadness,hate,hate,sadness,sadness,hate,hate


In [ ]:
predicted_df.to_csv(os.path.join(data_dir, "lyrics_validation_df_model_predicted.csv"))